## Deliverable 2. Create a Customer Travel Destinations Map.

In [26]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [27]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Albany,US,42.60,-73.97,66.99,76,68,1.99,broken clouds
1,1,Punta Arenas,CL,-53.15,-70.92,46.40,70,75,8.05,broken clouds
2,2,Ankang,CN,32.68,109.02,53.47,66,79,2.75,broken clouds
3,3,Thompson,CA,55.74,-97.86,24.80,58,75,8.05,light snow
4,4,Ketchikan,US,55.34,-131.65,46.40,28,1,3.36,clear sky


In [28]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
11,11,Taoudenni,ML,22.68,-3.98,87.01,13,18,10.76,few clouds
18,18,Kapaa,US,22.08,-159.32,79.00,88,90,10.29,light rain
27,27,Gamba,GA,-2.65,10.00,76.57,87,100,7.52,overcast clouds
30,30,Lintangan,PH,7.22,121.95,78.80,83,75,2.24,broken clouds
43,43,Georgetown,MY,5.41,100.34,78.80,94,20,3.71,few clouds
44,44,Hilo,US,19.73,-155.09,87.80,55,1,12.75,clear sky
48,48,Butaritari,KI,3.07,172.79,82.17,73,100,12.75,overcast clouds
53,53,Gorontalo,ID,0.54,123.06,79.97,75,33,0.83,scattered clouds
54,54,Avarua,CK,-21.21,-159.78,78.80,69,80,11.41,broken clouds
57,57,Kavaratti,IN,10.57,72.64,80.13,78,93,13.87,overcast clouds


In [31]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                207
City                   207
Country                207
Lat                    207
Lng                    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
Current description    207
dtype: int64

In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()



In [34]:
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
11,Taoudenni,ML,87.01,few clouds,22.68,-3.98,
18,Kapaa,US,79.00,light rain,22.08,-159.32,
27,Gamba,GA,76.57,overcast clouds,-2.65,10.00,
30,Lintangan,PH,78.80,broken clouds,7.22,121.95,
43,Georgetown,MY,78.80,few clouds,5.41,100.34,
44,Hilo,US,87.80,clear sky,19.73,-155.09,
48,Butaritari,KI,82.17,overcast clouds,3.07,172.79,
53,Gorontalo,ID,79.97,scattered clouds,0.54,123.06,
54,Avarua,CK,78.80,broken clouds,-21.21,-159.78,
57,Kavaratti,IN,80.13,overcast clouds,10.57,72.64,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
   
        

In [36]:
# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
 
    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
    
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(20)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
1,Vaini,TO,75.20,broken clouds,-21.20,-175.20,Keleti Beach Resort
3,Takoradi,GH,76.69,broken clouds,4.88,-1.76,Raybow International Hotel
7,Bengkulu,ID,81.01,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
8,Sur,OM,78.40,clear sky,22.57,59.53,Sur Plaza Hotel
11,Bathsheba,BB,82.40,broken clouds,13.22,-59.52,Atlantis Hotel
13,Butaritari,KI,82.00,few clouds,3.07,172.79,Isles Sunset Lodge
14,Mahebourg,MU,75.20,shower rain,-20.41,57.70,Shandrani Beachcomber Resort & Spa
25,Isangel,VU,81.23,light rain,-19.55,169.27,Volcano Sea View Tree House
26,Saint-Philippe,RE,75.20,broken clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
36,Kapaa,US,82.40,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach


In [37]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)



hotel_df.dropna(subset=['Hotel Name'], inplace=True)


In [38]:
hotel_df.head(20)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
18,Kapaa,US,79.00,light rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
43,Georgetown,MY,78.80,few clouds,5.41,100.34,Cititel Penang
44,Hilo,US,87.80,clear sky,19.73,-155.09,Hilo Hawaiian Hotel
48,Butaritari,KI,82.17,overcast clouds,3.07,172.79,Isles Sunset Lodge
53,Gorontalo,ID,79.97,scattered clouds,0.54,123.06,Maqna Hotel by Prasanthi
54,Avarua,CK,78.80,broken clouds,-21.21,-159.78,Paradise Inn
57,Kavaratti,IN,80.13,overcast clouds,10.57,72.64,Hotels in Lakshadweep Islands
66,Atuona,PF,78.26,few clouds,-9.80,-139.03,Villa Enata
74,Ranong,TH,76.46,overcast clouds,9.97,98.63,Tinidee Hotel@Ranong
78,Abu Dhabi,AE,80.60,clear sky,24.47,54.37,Beach Rotana


In [39]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"


In [40]:
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>

<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current description</dt><dd>{Current description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

</dl>
"""




In [42]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [43]:

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))